In [5]:
import pandas as pd
import re
import json
from bs4 import BeautifulSoup
import rdata
import numpy as np

In [6]:
data = rdata.read_rda(r'C:\Users\jaiga\Downloads\Json_SOPS\data\nacc_synth.rda')

In [7]:
data.keys()

dict_keys(['nacc_synth'])

In [8]:
nacc = data['nacc_synth']

In [9]:
nacc.shape

(261, 1708)

In [10]:
data['nacc_synth']['race'].value_counts()

race
1    67
2    55
Name: count, dtype: Int64

In [11]:
nacc.head(5)

,adc_sub_id,redcap_event_name,redcap_repeat_instance,subj_dob,subj_sex,subj_race,subj_age,enroll_dt,enroll_status,pet_partic,...,pax_dt,pbmc_count,pbmc_dt,plasma_count,plasma_dt,serum_count,serum_dt,urine_count,urine_dt,biospecimen_inventory_complete
1,ADC231,visit_info_arm_1,1,1964-11-18,Male,Black or African American,61,2021-06-24,Currently Enrolled,Consented and Enrolled,...,2022-12-19,1,2022-09-14,72,2022-02-10,7,2022-02-12,2,2022-01-09,2
2,ADC231,visit_info_arm_1,2,1960-05-06,Male,Black or African American,66,2021-05-10,Currently Enrolled,Consented and Enrolled,...,2022-12-07,1,2022-04-03,58,2022-11-16,6,2022-08-04,2,2022-06-27,2
3,ADC225,visit_info_arm_1,1,1961-06-28,Female,White,59,2020-02-18,Currently Enrolled,Consented and Enrolled,...,2022-02-28,1,2022-12-08,30,2022-09-05,7,2022-11-20,1,2022-01-27,2
4,ADC239,visit_info_arm_1,1,1933-01-16,Female,Black or African American,74,2021-11-20,Currently Enrolled,Consented and Enrolled,...,2022-04-16,2,2022-09-12,59,2022-06-09,7,2022-08-03,1,2022-12-21,2
5,ADC194,visit_info_arm_1,1,1958-06-24,Female,Black or African American,76,2021-09-16,Unable to Complete,Consented and Enrolled,...,2022-08-09,1,2022-02-19,30,2022-07-15,18,2022-09-08,1,2022-01-27,2


In [12]:
nacc['raceter']

1        88
2      <NA>
3        88
4        88
5        88
       ... 
257    <NA>
258    <NA>
259    <NA>
260    <NA>
261    <NA>
Name: raceter, Length: 261, dtype: Int32

In [13]:
nacc['raceter'].value_counts(dropna=False)

raceter
<NA>    139
88      121
99        1
Name: count, dtype: Int64

In [14]:
nacc['racesec'].value_counts(dropna=False)

racesec
<NA>    139
88      120
99        2
Name: count, dtype: Int64

In [15]:
nacc['race'].value_counts(dropna=False)

race
<NA>    139
1        67
2        55
Name: count, dtype: Int64

In [16]:
nacc['race'].head(25)

1        1
2     <NA>
3        1
4        1
5        1
6        2
7     <NA>
8        1
9        2
10       1
11       1
12    <NA>
13       1
14    <NA>
15       2
16       2
17    <NA>
18       2
19    <NA>
20       2
21       1
22       2
23    <NA>
24       2
25       1
Name: race, dtype: Int32

In [17]:
nacc1 = nacc.copy()

In [18]:
# Randomly select indices
random_indices = [2, 7,12,17,19]
# Impute selected rows with 2
nacc1.loc[random_indices, 'racesec'] = 2

In [19]:
nacc1['hispanic'].value_counts(dropna=False)

hispanic
<NA>    140
0       120
9         1
Name: count, dtype: Int64

In [20]:
# Randomly select indices
random_indices = [14, 6, 19,23]
# Impute selected rows with 2
nacc1.loc[random_indices, 'raceter'] = 2

In [21]:
nacc1['raceter'].value_counts(dropna=False)

raceter
<NA>    136
88      120
2         4
99        1
Name: count, dtype: Int64

In [22]:
# Load JSON mapping file
with open(r'C:\Users\jaiga\Downloads\crosswalks\A1_UDS4_programmatic_crosswalk_mappings.json', 'r', encoding='utf-8') as file:
    mapping_data = json.load(file)

In [23]:
mapping_data.keys()

dict_keys(['Direct_Mappings', 'Conditional_Consistency', 'Structured_Transformations', 'High_Complexity'])

In [24]:
# Load UDS3 data with UTF-8 encoding
uds3_df = nacc1  # Ensure nacc is defined before executing
uds3_df.columns = uds3_df.columns.str.lower()
uds4_df = pd.DataFrame()

In [32]:
# Function to process mappings dynamically
def process_mappings(mapping_type):
    for mapping in mapping_data.get(mapping_type, []):
        uds3_var = mapping["UDS3_variable"].lower()  # Convert to lowercase to match DataFrame columns
        uds4_var = mapping["UDS4_variable"].lower()
        
        if mapping_type == 'Direct_Mappings':
            # Extract different mapping types
            response_levels = [m for m in mapping["crosswalk_remappings"] if m["mapping_type"] == "Response LEVELS"]
            conformity = [m for m in mapping["crosswalk_remappings"] if m["mapping_type"] == "Conformity"]
            
        else:
            # Extract different mapping types
            response_levels = [m for m in mapping["crosswalk_mappings"] if m["mapping_type"] == "Response LEVELS"]
            conformity = [m for m in mapping["crosswalk_mappings"] if m["mapping_type"] == "Conformity"]
        
        # Case 1: No Response LEVELS, copy values directly
        if not response_levels:
            if uds3_var in uds3_df.columns:
                uds4_df[uds4_var] = uds3_df[uds3_var]
        
        # Case 2: Conformity check
        elif conformity:
            uds3_conformity_values = set(item["UDS3_value"] for mapping in conformity for item in mapping["mappings"])
            uds4_conformity_values = set(item["UDS4_value"] for mapping in conformity for item in mapping["mappings"])
            
            if uds3_conformity_values == uds4_conformity_values:  # Ensure conformity matches
                if uds3_var in uds3_df.columns:
                    uds4_df[uds4_var] = uds3_df[uds3_var]
            else:
                response_map = {str(item["UDS3_value"]): str(item["UDS4_value"]) for mapping in response_levels for item in mapping["mappings"]}
                if uds4_var == 'raceblack':
                    print(response_map)
                if uds3_var in uds3_df.columns:
                    for uds3_value, uds4_value in response_map.items():
                        if "|" not in uds3_value:  # Handle individual mappings first
                            uds4_df.loc[uds3_df[uds3_var].astype(str) == uds3_value, uds4_var] = uds4_value
                        else:
                            if not (uds3_var.endswith("sec") or uds3_var.endswith("ter")):
                                values = uds3_value.split(" | ")
                                uds4_df.loc[uds3_df[uds3_var].astype(str).isin(values), uds4_var] = uds4_value
                

# Process different mapping types dynamically
mapping_types = ["Direct_Mappings", "Conditional_Consistency"]  # Add more types as needed
for mapping_type in mapping_types:
    process_mappings(mapping_type)

# Save transformed UDS4 data with UTF-8-SIG encoding to avoid character corruption
uds4_df.to_csv("uds4_data1.csv", index=False, na_rep="NA", encoding='utf-8-sig')


In [33]:
uds4_df.raceblack.value_counts()

raceblack
None    67
1       62
Name: count, dtype: int64

### Testing

In [27]:
for mapping in mapping_data.get(mapping_type, []):
    uds3_var = mapping["UDS3_variable"].lower()  # Convert to lowercase to match DataFrame columns
    uds4_var = mapping["UDS4_variable"].lower()
    response_levels = [m for m in mapping["crosswalk_mappings"] if m["mapping_type"] == "Response LEVELS"]
    
    if not response_levels:
        if uds3_var in uds3_df.columns:
            uds4_df[uds4_var] = uds3_df[uds3_var]
    
    response_map = {str(item["UDS3_value"]): str(item["UDS4_value"]) for mapping in response_levels for item in mapping["mappings"]}
    
    if (uds4_var == 'raceblack'):
        print(response_map)

                    
print(uds4_df['raceblack'].value_counts(dropna=False))

{'2': '1', '1 | 3 | 4 | 5 | 50 | 99': 'None'}
{'2': '1', '1 | 3 | 4 | 5 | 50 | 88 | 99': 'None'}
{'2': '1', '1 | 3 | 4 | 5 | 50 | 88 | 99': 'None'}
raceblack
NaN     132
None     67
1        62
Name: count, dtype: int64


In [28]:
uds4_df.head(5)

,birthmo,educ,handed,livsitua,maristat,refersc,birthsex,sourcenw,zip,birthyr,ethispanic,predomlan,raceaian,raceasian,raceblack,racenhpi,raceunkn,racewhite,residenc
1,6,18,2,3,1,6,6,1,352,1951,None,1,None,None,None,None,None,1,1
2,11,<NA>,<NA>,2,2,<NA>,5,<NA>,350,1963,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,1
3,3,18,2,1,1,1,6,1,353,1956,None,1,None,None,None,None,None,1,1
4,8,16,2,2,1,4,6,1,350,1963,None,1,None,None,None,None,None,1,1
5,4,20,2,2,1,8,5,1,350,1961,None,1,None,None,None,None,None,1,1


In [29]:
uds4_df.raceblack.value_counts(dropna=False)

raceblack
NaN     132
None     67
1        62
Name: count, dtype: int64

In [30]:
uds4_df.racewhite.value_counts(dropna=False)

racewhite
NaN     139
1        67
None     55
Name: count, dtype: int64